<a href="https://colab.research.google.com/github/andresfelipecs/suena_application_test/blob/master/algotrading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing libraries

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.subplots as sp
from statsmodels.tsa.statespace.sarimax import SARIMAX
import statsmodels.api as sm
import tensorflow as tf
import time
from statsmodels.graphics.tsaplots import plot_pacf, plot_acf
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, Input, LSTM
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model, Sequential
from sklearn.model_selection import TimeSeriesSplit
from itertools import product
import warnings
from scipy.stats.distributions import norm
import xgboost as xgb
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_error
from hyperopt import fmin, tpe, hp
from functools import partial
from typing import List, Tuple
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score



# Loading data

In [ ]:
# Loading the data into a Pandas DataFrame
df_auction = pd.read_csv("/content/sample_data/auction_data.csv", sep=';', header=0)

# Setting the first column as an index 
df_auction['Date (WET)'] = pd.to_datetime(df_auction['Date (WET)'], format='[%d/%m/%Y %H:%M]')
# df_auction['date'] = df_auction['Date (WET)']
df_auction.set_index('Date (WET)', inplace=True)


# Droping the first row
units_auction = df_auction.iloc[0]
print(units_auction)

df_auction = df_auction.drop(df_auction.index[0])


# Showing the data 
print(df_auction)
# print(f'INDEX:  {df_auction.index} \n')
# print(f'TYPE:  {df_auction.dtypes} \n')
# print(f'SHAPE:  {df_auction.shape} \n')
print(f"COLUMNS: {df_auction.columns} \n")

# print(f'ROW {df_auction.loc["2021-01-01 00:00:00" ]}')


# Converting all the elements in the df_auction dataframe to numerical values, so that the data can be used for numerical computations and visualizations.
df_auction = df_auction.apply(pd.to_numeric, errors='coerce')
# The errors='coerce' argument is used to handle any elements that can't be converted to numerical values. 
# When errors='coerce', any non-numerical values will be replaced with NaN (Not a Number) values.




In [ ]:
fig = make_subplots(
    rows=3, cols=1,
    shared_xaxes=True,
    vertical_spacing=0.03,
    specs=[[{"type": "table"}],
           [{"type": "scatter"}],
           [{"type": "scatter"}]]
)

fig.add_trace(
    go.Scatter(
        x=df_auction.index,
        y=df_auction["price_first_auction"],
        mode="lines",
        name="Price first auction"
    ),
    row=2, col=1
)

fig.add_trace(
    go.Scatter(
        x=df_auction.index,
        y=df_auction["price_second_auction"],
        mode="lines",
        name="Price second auction"
    ),
    row=3, col=1
)

fig.add_trace(
    go.Table(
        header=dict(
            values=["price first auction", "Price second auction", "Traded volume first_auction",
                    "Traded volume second auction", "Price forecast first auction"],
            font=dict(size=10),
            align="left"
        ),
        cells=dict(
            values=[df_auction[k].tolist() for k in df_auction.columns[:]],
            align = "left")
    ),
    row=1, col=1
)
fig.update_layout(
    height=800,
    showlegend=True,
    title_text="df_auction",
)

fig.show()

In [ ]:
df_forecast_inputs = pd.read_csv(
            "/content/sample_data/forecast_inputs.csv"
        , sep=';', header=0)

# Setting the first column as an index 
df_forecast_inputs['Date (WET)'] = pd.to_datetime(df_forecast_inputs['Date (WET)'], format='[%d/%m/%Y %H:%M]')
df_forecast_inputs.set_index('Date (WET)', inplace=True)

# Dropping the first row
units_forecast_inputs = df_forecast_inputs.iloc[0]
print(units_forecast_inputs)

df_forecast_inputs = df_forecast_inputs.drop(df_forecast_inputs.index[0])

df_forecast_inputs = df_forecast_inputs.apply(pd.to_numeric, errors='coerce')


print(df_forecast_inputs)




In [ ]:
fig = make_subplots(
    rows=3, cols=1,
    shared_xaxes=True,
    vertical_spacing=0.03,
    specs=[[{"type": "table"}],
           [{"type": "table"}],
           [{"type": "table"}]]
)

fig.add_trace(
    go.Table(
        header=dict(
            values=["Demand plus system margin (MW)", "Demand (MW)", "Within day availability (MW)",
                    "Margin (MW)", "within_day_margin (MW)", "Long term wind (GBP/MWh)"],
            font=dict(size=10),
            align="left"
        ),
        cells=dict(
            values=[df_forecast_inputs[k].tolist() for k in df_forecast_inputs.columns[:6]],
            align = "left")
    ),
    row=1, col=1
)
                  

fig.add_trace(
    go.Table(
        header=dict(
            values=["Long term solar (MW)", "Long term wind over demand (%)", "long term wind over margin (%)",
                    "long term solar over demand (%)", "long term solar over margin (%)", "margin over demand (%)"],
            font=dict(size=10),
            align="left"
        ),
        cells=dict(
            values=[df_forecast_inputs[k].tolist() for k in df_forecast_inputs.columns[6:12]],
            align = "left")
    ),
    row=2, col=1
)


fig.add_trace(
    go.Table(
        header=dict(
            values=["Snsp forecast (%)", "Stack price (GBP/MWh)", "Within day potential stack price (GBP/MWh)",
                    "previous day ahead price (GBP/MWh)", "previous continuous half hour vwap (GBP/MWh)", "inertia forecast (GVA.s)"],
            font=dict(size=10),
            align="left"
        ),
        cells=dict(
            values=[df_forecast_inputs[k].tolist() for k in df_forecast_inputs.columns[12:]],
            align = "left")
    ),
    row=3, col=1
)
fig.update_layout(
    height=800,
    showlegend=False,
    title_text="df_forecast_inputs",
)

fig.show()

In [ ]:
df_system_prices = pd.read_csv(
            "/content/sample_data/system_prices.csv"
        , sep=';', header=0)

# Setting the first column as an index 
df_system_prices['Date (WET)'] = pd.to_datetime(df_system_prices['Date (WET)'], format='[%d/%m/%Y %H:%M]')
df_system_prices.set_index('Date (WET)', inplace=True)

# Dropping the first row
units_system_prices = df_system_prices.iloc[0]
print(units_system_prices)

df_system_prices = df_system_prices.drop(df_system_prices.index[0])

df_system_prices = df_system_prices.apply(pd.to_numeric, errors='coerce')


print(df_system_prices)



In [ ]:
fig = make_subplots(
    rows=4, cols=1,
    shared_xaxes=True,
    vertical_spacing=0.03,
    specs=[[{"type": "table"}],
           [{"type": "scatter"}],
           [{"type": "scatter"}],
           [{"type": "scatter"}]
           ]
)

fig.add_trace(
    go.Scatter(
        x=df_system_prices.index,
        y=df_system_prices["forecast_system_price_high"],
        mode="lines",
        name="Forecast system price high"
    ),
    row=4, col=1
)

fig.add_trace(
    go.Scatter(
        x=df_system_prices.index,
        y=df_system_prices["forecast_system_price_low"],
        mode="lines",
        name="Forecast system price low"
    ),
    row=3, col=1
)

fig.add_trace(
    go.Scatter(
        x=df_system_prices.index,
        y=df_system_prices["system_price"],
        mode="lines",
        name="System price "
    ),
    row=2, col=1
)

fig.add_trace(
    go.Table(
        header=dict(
            values=["Forecast system price low GBP/MWh", "Forecast system price high GBP/MWh", "System price GBP/MWh"],
            font=dict(size=10),
            align="left"
        ),
        cells=dict(
            values=[df_system_prices[k].tolist() for k in df_system_prices.columns[:]],
            align = "left")
    ),
    row=1, col=1
)

fig.update_layout(
    height=800,
    showlegend=True,
    title_text="df_system_prices",
)

fig.show()

# Cleaning and Splitting the data into training and testing sets

In [ ]:

# Filling NaN with mode 
df_auction.fillna(df_auction.mode().iloc[0], inplace=True)
df_forecast_inputs.fillna(df_forecast_inputs.mode().iloc[0], inplace=True)
df_system_prices.fillna(df_system_prices.mode().iloc[0], inplace=True)

train_auction = df_auction[df_auction.index < '2022-03-01']
train_auction = train_auction.apply(pd.to_numeric, errors='coerce')

test_auction = df_auction[df_auction.index >= '2022-03-01']
test_auction = test_auction.apply(pd.to_numeric, errors='coerce')

train_forecast_inputs = df_forecast_inputs[df_forecast_inputs.index < '2022-03-01']
train_forecast_inputs = train_forecast_inputs.apply(pd.to_numeric, errors='coerce')

test_forecast_inputs = df_forecast_inputs[df_forecast_inputs.index >= '2022-03-01']
test_forecast_inputs = test_forecast_inputs.apply(pd.to_numeric, errors='coerce')

train_system_prices = df_system_prices[df_system_prices.index < '2022-03-01']
train_system_prices = train_system_prices.apply(pd.to_numeric, errors='coerce')

test_system_prices = df_system_prices[df_system_prices.index >= '2022-03-01']
test_system_prices = test_system_prices.apply(pd.to_numeric, errors='coerce')

# Using scaler to normalize ALL the data 
scaler = StandardScaler()

all_data = pd.concat([df_auction, df_forecast_inputs, df_system_prices], axis=1)
all_data = all_data.apply(pd.to_numeric, errors='coerce')
regression_all_data = all_data
all_data = pd.DataFrame(scaler.fit_transform(all_data), columns=all_data.columns, index=all_data.index)


normalized_train_data = all_data[all_data.index < '2022-03-01']
normalized_test_data = all_data[all_data.index >= '2022-03-01']

normalized_train_auction = normalized_train_data[df_auction.columns]
normalized_test_auction = normalized_test_data[df_auction.columns]

normalized_train_forecast_inputs = normalized_train_data[df_forecast_inputs.columns]
normalized_test_forecast_inputs = normalized_test_data[df_forecast_inputs.columns]

normalized_train_system_prices = normalized_train_data[df_system_prices.columns]
normalized_test_system_prices = normalized_test_data[df_system_prices.columns]

print(normalized_train_auction)

# Making sure there are not NaN or inf values 
print(f"train auction null data {train_auction.isnull().sum().sum()}")
print(f"train auction inf data {np.isinf(train_auction).sum().sum()}")

print(f"train forecast input null data {train_forecast_inputs.isnull().sum().sum()}")
print(f"tran forecast input inf data {np.isinf(train_forecast_inputs).sum().sum()}")


print(f"train system price null data {train_system_prices.isnull().sum().sum()}")
print(f"traion system price inf data {np.isinf(train_system_prices).sum().sum()}")

print(train_auction.columns)
print(train_forecast_inputs.columns)
print(train_system_prices.columns)




# Regression analysis using scikit-learn

In [ ]:


# Assuming your DataFrame is named 'regression_all_data'
df = regression_all_data

# Prepare your independent (X) and dependent (y) variables
X = df.drop(['price_first_auction', 'price_second_auction'], axis=1)
y = df['price_first_auction']  # Change to 'price_second_auction' if needed

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create and train the linear regression model
regression_model = LinearRegression()
regression_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = regression_model.predict(X_test)

# Evaluate the performance of the model using metrics like Mean Squared Error (MSE) and R-squared (R²)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error:", mse)
print("R-squared:", r2)

# Check the coefficients and intercept of the model
print("Coefficients:", regression_model.coef_)
print("Intercept:", regression_model.intercept_)


### Linear Regression Results

We performed a linear regression analysis to predict the `price_first_auction` using the given independent variables. Below are the key findings from our analysis:

**Model Performance**

* **Mean Squared Error (MSE):** 2917.22
* **R-squared (R²):** 0.7875

Our model achieved an R-squared value of 0.7875, which means that it explains approximately 78.75% of the variability in the `price_first_auction`. However, it's important to compare this performance to other models or a baseline model to better understand its relative performance.

**Model Coefficients**

The coefficients of our linear regression model provide insights into the relationships between the independent variables and the dependent variable (`price_first_auction`). The table below summarizes the coefficients:





In [ ]:
# Create a DataFrame with the coefficients
coefficients_df = pd.DataFrame({
    'Independent Variable': [
        'traded_volume_first_auction',
        'traded_volume_second_auction',
        'price_forecast_first_auction',
        'demand_plus_system_margin',
        'demand',
        'within_day_availability',
        'margin',
        'within_day_margin',
        'long_term_wind',
        'long_term_solar',
        'long_term_wind_over_demand',
        'long_term_wind_over_margin',
        'long_term_solar_over_demand',
        'long_term_solar_over_margin',
        'margin_over_demand',
        'snsp_forecast',
        'stack_price',
        'within_day_potential_stack_price',
        'previous_day_ahead_price',
        'previous_continuous_half_hour_vwap',
        'inertia_forecast',
        'forecast_system_price_low',
        'forecast_system_price_high',
        'system_price'
    ],
    'Coefficient': regression_model.coef_
})

# Display the coefficients DataFrame
pd.set_option('display.float_format', '{:.4e}'.format)  # Displayed coefficients in scientific notation
display(coefficients_df)


A positive coefficient indicates that the `price_first_auction` increases as the independent variable increases, while a negative coefficient suggests the opposite. For example, the coefficient of `traded_volume_first_auction` is -0.0014, which means that as it increases by one unit, the `price_first_auction` decreases by about 0.0014 units, holding all other variables constant.

**Model Intercept**

The intercept of our model is 14.3816. This value represents the expected `price_first_auction` when all independent variables are equal to zero.

---

In conclusion, our linear regression model provides a reasonable understanding of the relationships between the independent variables and the `price_first_auction`. However, it's crucial to validate the assumptions of linear regression, perform additional diagnostics, and compare the model's performance to other models or a baseline model before drawing final conclusions.


In [ ]:
columns_to_plot = [
    'price_first_auction',
    'price_second_auction',
    'traded_volume_first_auction',
    'traded_volume_second_auction',
    'price_forecast_first_auction',
    'demand_plus_system_margin',
    'demand',
    'within_day_availability',
    'margin',
    'within_day_margin',
    'long_term_wind',
    'long_term_solar',
    'long_term_wind_over_demand',
    'long_term_wind_over_margin',
    'long_term_solar_over_demand',
    'long_term_solar_over_margin',
    'margin_over_demand',
    'snsp_forecast',
    'stack_price',
    'within_day_potential_stack_price',
    'previous_day_ahead_price',
    'previous_continuous_half_hour_vwap',
    'inertia_forecast',
    'forecast_system_price_low',
    'forecast_system_price_high',
    'system_price'
]

# Creating the scatter plot matrix
fig = px.scatter_matrix(regression_all_data[columns_to_plot])

fig.update_traces(marker=dict(size=3, opacity=0.5))
fig.update_layout(width=1500, height=1500, title='Scatter Plot Matrix of Variables')

fig.update_xaxes(tickfont=dict(size=1))
fig.update_yaxes(tickfont=dict(size=1))

fig.show()

In [ ]:
# Prepare your independent (X) and dependent (y) variables
X = regression_all_data.drop(['price_first_auction', 'price_second_auction'], axis=1)
y = regression_all_data['price_first_auction']  # Change to 'price_second_auction' if needed

# Create a subplot for each independent variable
fig = sp.make_subplots(rows=len(X.columns), cols=1, subplot_titles=X.columns)

# Iterate through each independent variable, create a scatter plot, and add the regression line
for i, column in enumerate(X.columns):
    X_single = X[[column]]
    
    model = LinearRegression()
    model.fit(X_single, y)
    
    x_range = np.linspace(X[column].min(), X[column].max(), 100).reshape(-1, 1)
    y_range = model.predict(x_range)
    
    scatter = go.Scatter(x=X[column], y=y, mode='markers', name=column, marker=dict(size=3, opacity=0.65))
    reg_line = go.Scatter(x=x_range.ravel(), y=y_range, name=f"Regression Fit - {column}", line=dict(color='red'))

    fig.add_trace(scatter, row=i+1, col=1)
    fig.add_trace(reg_line, row=i+1, col=1)

# Update the layout
fig.update_layout(height=300*len(X.columns), width=1000, title='Scatter Plots with Regression Lines for Independent Variables')

# Show the plot
fig.show()

# Checking the shapes of the resulting training and testing sets





In [ ]:

print("train_auction shape: ", train_auction.shape)
print("test_auction shape: ", test_auction.shape)

print("train_forecast_inputs shape: ", train_forecast_inputs.shape)
print("test_forecast_inputs shape: ", test_forecast_inputs.shape)

print("train_system_prices shape: ", train_system_prices.shape)
print("test_system_prices shape: ", test_system_prices.shape)

# Tensor flow & keras: predictions with Kfold validations (1D Convolutional Neural Network (CNN))

In [ ]:

# Splitting the input data into features and target
train_features = normalized_train_auction.drop('price_second_auction', axis=1)
train_target = normalized_train_auction['price_second_auction']
# Concatenating the other datasets with the training features
train_features = np.concatenate((train_features, normalized_train_forecast_inputs, normalized_train_system_prices), axis=1)

# Defining the number of folds for the K-fold cross-validation
n_folds = 5

# Initializing a KFold object with the number of folds
kfold = KFold(n_splits=n_folds, shuffle=True)

# Initializing arrays to store the results of the cross-validation
cv_scores = []
cv_histories = []

# Looping over the folds
for train_idx, val_idx in kfold.split(train_features):
    # Splitting the data into training and validation sets for each fold
    x_train, x_val = train_features[train_idx], train_features[val_idx]
    y_train, y_val = train_target[train_idx], train_target[val_idx]
    
    # Building the neural network model using TensorFlow
    model = tf.keras.models.Sequential()
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(x_train.shape[1], 1)))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1))

    # Compiling the model
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error', metrics=['mean_absolute_error'])

    # Defining callbacks to monitor the training process
    callbacks = [
        EarlyStopping(monitor='val_loss', patience=10),
        ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', save_best_only=True),
        ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=0.0001)
    ]

    # Reshaping the data to include a channel dimension
    x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], 1)
    x_val = x_val.reshape(x_val.shape[0], x_val.shape[1], 1)

    # Training the model on the training data for each fold
    history = model.fit(x_train, y_train, epochs=100, batch_size=32, validation_data=(x_val, y_val), callbacks=callbacks)

    # Evaluating the model on the validation data for each fold
    score = model.evaluate(x_val, y_val, verbose=0)

    # Appending the results of the fold to the arrays
    cv_scores.append(score)
    cv_histories.append(history)

# Calculating the average performance of the model across all folds
mean_score = np.mean(cv_scores, axis=0)

# Printing the average performance of the model
print("Average performance on validation data:")
print("Loss: ", mean_score[0])
print("Mean Absolute Error: ", mean_score[1])


# Using the trained model to make predictions on the train data
validated_train_forecast = model.predict(train_features)

print(validated_train_forecast)
print(validated_train_forecast.shape)

# Use the trained model to make predictions on the test data
test_features = normalized_test_auction.drop('price_second_auction', axis=1)

# Concatenate the other datasets with the test features
test_features = np.concatenate((test_features, normalized_test_forecast_inputs, normalized_test_system_prices), axis=1)

validated_test_forecast = model.predict(test_features)


# Summary: 
# The model being used is a 1D Convolutional Neural Network (CNN) built using TensorFlow. 
# It has multiple layers, including two 1D convolutional layers, two max pooling layers, a flatten layer, two dense layers, and a dropout layer. 
# It's trained using the Adam optimizer with a learning rate of 0.001 and the mean squared error (MSE) loss function. 
# It's also monitored using several callbacks, including EarlyStopping, ModelCheckpoint, and ReduceLROnPlateau. 
# Performming K-fold cross-validation, where the data is split into training and validation sets for each fold, and the model is trained and evaluated on each fold. 
# Finally, the trained model is used to make predictions on the train and test data.

# Decided to change the model to CNN, one most commonly used for time series forecasting to validate and compare results with the previous model.
# It is also recommended to do K-fold cross-validation in order to have a better performace estimation, reduce overfitting and helps us identifying 
# the model stability. If the performance scores low variability across different folds, it suggests that the model may perform well on unseen data.

# This model approach, as the previous one, is using the train_features wihtout the second_price_auction to find the train_forecast
#  and the test_features wihtout the second_price_auction to find the test_forecast


## Performance scores

In [ ]:
# Calculate the mean and standard deviation of the target variable
target_mean = np.mean(normalized_train_auction['price_second_auction'])
target_std = np.std(normalized_train_auction['price_second_auction'])

# Print the results
print("Target Mean: ", target_mean)
print("Target Standard Deviation: ", target_std)

# Print the average performance of the model
print("Average performance on validation data:")
print("Loss: ", mean_score[0])
print("Mean Absolute Error: ", mean_score[1])

# Based on the performance metrics, the model is performing relatively well. 
# The average loss value of 0.064 and mean absolute error value of 0.107 
# It indicates that the model is making predictions that are relatively close to the true values, 
# with an average absolute difference of approximately 0.11 between the predicted and true values.


## Results plot: Plotting validated normalized values 

In [ ]:
print(validated_train_forecast)
print(validated_test_forecast)

# Creating a dataframe for the train forecast
validated_train_forecast_df = pd.DataFrame(validated_train_forecast, index=normalized_train_auction.index, columns=['price_second_auction_forecast'])

# Creating a dataframe for the test forecast
validated_test_forecast_df = pd.DataFrame(validated_test_forecast, index=normalized_test_auction.index, columns=['price_second_auction_forecast'])

fig = make_subplots()

fig.add_trace(go.Scatter(x=normalized_train_auction.index, y=normalized_train_auction['price_second_auction'], name='Train Price Second Auction', line=dict(color='#80b1d3')))
fig.add_trace(go.Scatter(x=validated_train_forecast_df.index, y=validated_train_forecast_df['price_second_auction_forecast'], name='Train Price Second Auction Forecast', line=dict(color='#fdb462', width=0.8)))

fig.add_trace(go.Scatter(x=normalized_test_auction.index, y=normalized_test_auction['price_second_auction'], name='Test Price Second Auction', line=dict(color='#fb8072')))
fig.add_trace(go.Scatter(x=validated_test_forecast_df.index, y=validated_test_forecast_df['price_second_auction_forecast'], name='Test Price Second Auction Forecast', line=dict(color='#8dd3c7', width=0.8)))

fig.update_layout(title='Normalized Train and Test Forecasts Second Auction')
fig.update_xaxes(title='Date')
fig.update_yaxes(title='Price')

fig.show()

## Plotting train **prices** and **volume** 

In [ ]:


fig = make_subplots(
            rows=2,
            cols=2,
            shared_xaxes=True,
            vertical_spacing=0.08,
            subplot_titles=("1st Action Prices", "2nd Action Prices", "1st Action Volume", "2nd Action Volume"),
            row_width=[0.2, 0.7],
        )

fig.add_trace(
            go.Scatter(x=normalized_train_auction.index, y=normalized_train_auction["price_first_auction"], mode="lines", line_shape="spline", name="Train Price First Auction", line=dict(color='#80b1d3')),
            row=1,
            col=1,
        )
fig.add_trace(
            go.Scatter(x=normalized_train_auction.index, y=normalized_train_auction["price_second_auction"], mode="lines", line_shape="spline", name="Train Price Second Auction", line=dict(color='#fb8072')),
            row=1,
            col=2,
        )

fig.add_trace(
            go.Bar(
                x=normalized_train_auction.index,
                y=normalized_train_auction["traded_volume_first_auction"],
                showlegend=False,
                marker=dict(
                    opacity=1,
                    line=dict(width=2, color="#ffffb3"),
                ),
            ),
            row=2,
            col=1,
        )

fig.add_trace(
            go.Bar(
                x=normalized_train_auction.index,
                y=normalized_train_auction["traded_volume_second_auction"],
                showlegend=False,
                marker=dict(
                    opacity=1,
                    line=dict(width=2, color="#ffffb3"),
                ),
            ),
            row=2,
            col=2,
        )

fig.add_trace(
            go.Scatter(
                x=normalized_train_auction.index,
                y=normalized_train_auction["price_forecast_first_auction"],
                mode="lines",
                name="Price train forecast first auction",
                line=dict(color="#fdb462", width=0.5),
            ),
            row=1,
            col=1,
        )

fig.add_trace(
            go.Scatter(
                x=validated_train_forecast_df.index,
                y=validated_train_forecast_df['price_second_auction_forecast'],
                mode="lines",
                name="Price train forecast second auction",
                line=dict(color="#80b1d3", width=0.5),
            ),
            row=1,
            col=2,
        )
            
fig.update_layout(title='Normalized Training Actions Prices and Volumes')
fig.update_yaxes(title='Price', row=1)
fig.update_xaxes(title='Date', row=2)

fig.show()

## Plotting normalized test **prices** and **volume** 

In [ ]:

fig = make_subplots(
            rows=2,
            cols=2,
            shared_xaxes=True,
            vertical_spacing=0.08,
            subplot_titles=("1st Action Prices", "2nd Action Prices", "1st Action Volume", "2nd Action Volume"),
            row_width=[0.2, 0.7],
        )

fig.add_trace(
            go.Scatter(x=normalized_test_auction.index, y=normalized_test_auction["price_first_auction"], mode="lines", line_shape="spline",name="Test Price First Auction ", line=dict(color='#80b1d3')),
            row=1,
            col=1,
        )
fig.add_trace(
            go.Scatter(x=normalized_test_auction.index, y=normalized_test_auction["price_second_auction"], mode="lines", line_shape="spline",name="Test Price Second Auction", line=dict(color='#fb8072')),
            row=1,
            col=2,
        )

fig.add_trace(
            go.Bar(
                x=normalized_test_auction.index,
                y=normalized_test_auction["traded_volume_first_auction"],
                
                showlegend=False,
                marker=dict(
                    opacity=1,
                    line=dict(width=2, color="#ffffb3"),
                ),
            ),
            row=2,
            col=1,
        )

fig.add_trace(
            go.Bar(
                x=normalized_test_auction.index,
                y=normalized_test_auction["traded_volume_second_auction"],
                
                showlegend=False,
                marker=dict(
                    opacity=1,
                    line=dict(width=2, color="#ffffb3"),
                ),
            ),
            row=2,
            col=2,
        )

fig.add_trace(
            go.Scatter(
                x=normalized_test_auction.index,
                y=normalized_test_auction["price_forecast_first_auction"],
                mode="lines",
                name="Price train forecast first auction",
                line=dict(color="#fdb462", width=0.5),
            ),
            row=1,
            col=1,
        )

fig.add_trace(
            go.Scatter(
                x=validated_test_forecast_df.index,
                y=validated_test_forecast_df['price_second_auction_forecast'],
                mode="lines",
                name="Price train forecast second auction",
                line=dict(color="#80b1d3", width=0.5),
            ),
            row=1,
            col=2,
        )

fig.update_layout(title='Normalized Test Actions Prices and Volumes', showlegend=True)
fig.update_yaxes(title='Price', row=1)
fig.update_xaxes(title='Date', row=2)

fig.show()

##  Denormalized Plots: (Results) (train **prices** and **volume**) (test **prices** and **volume**)

In [ ]:

# Getting the mean and standard deviation used during normalization
mean = scaler.mean_[0]
std = scaler.scale_[0]

# Denormalizing the train forecast
validated_train_forecast_df['price_second_auction_forecast'] = validated_train_forecast_df['price_second_auction_forecast'] * std + mean
# Denormalizing the test forecast
validated_test_forecast_df['price_second_auction_forecast'] = validated_test_forecast_df['price_second_auction_forecast'] * std + mean

print(f"train forecast df : {validated_train_forecast_df['price_second_auction_forecast']}")
print(f"test forecast df {validated_test_forecast_df['price_second_auction_forecast']}")





In [ ]:
# print(train_auction)
# print(test_auction)

fig = make_subplots()

fig.add_trace(go.Scatter(x=train_auction.index, y=train_auction['price_second_auction'], name='Train Price Second Auction', line=dict(color='#80b1d3')))
fig.add_trace(go.Scatter(x=validated_train_forecast_df.index, y=validated_train_forecast_df['price_second_auction_forecast'], name='Train Price Second Auction Forecast', line=dict(color='#fdb462', width=0.5)))

fig.add_trace(go.Scatter(x=test_auction.index, y=test_auction['price_second_auction'], name='Test Price Second Auction', line=dict(color='#fb8072')))
fig.add_trace(go.Scatter(x=validated_test_forecast_df.index, y=validated_test_forecast_df['price_second_auction_forecast'], name='Test Price Second Auction Forecast', line=dict(color='#8dd3c7', width=0.5)))

fig.update_layout(title='Denormalized Train and Test Forecasts Second Auction ')
fig.update_xaxes(title='Date')
fig.update_yaxes(title='Price')

fig.show()



In [ ]:

fig = make_subplots(
            rows=2,
            cols=2,
            shared_xaxes=True,
            vertical_spacing=0.08,
            subplot_titles=("1st Action Prices", "2nd Action Prices", "1st Action Volume", "2nd Action Volume"),
            row_width=[0.2, 0.7],
        )

fig.add_trace(
            go.Scatter(x=train_auction.index, y=train_auction["price_first_auction"], mode="lines", line_shape="spline", name="Train Price First Auction", line=dict(color='#80b1d3')),
            row=1,
            col=1,
        )
fig.add_trace(
            go.Scatter(x=train_auction.index, y=train_auction["price_second_auction"], mode="lines", line_shape="spline", name="Train Price Second Auction", line=dict(color='#fb8072')),
            row=1,
            col=2,
        )

fig.add_trace(
            go.Bar(
                x=train_auction.index,
                y=train_auction["traded_volume_first_auction"],
                showlegend=False,
                marker=dict(
                    opacity=1,
                    line=dict(width=2, color="#ffffb3"),
                ),
            ),
            row=2,
            col=1,
        )

fig.add_trace(
            go.Bar(
                x=train_auction.index,
                y=train_auction["traded_volume_second_auction"],
                showlegend=False,
                marker=dict(
                    opacity=1,
                    line=dict(width=2, color="#ffffb3"),
                ),
            ),
            row=2,
            col=2,
        )

fig.add_trace(
            go.Scatter(
                x=train_auction.index,
                y=train_auction["price_forecast_first_auction"],
                mode="lines",
                name="Price train forecast first auction",
                line=dict(color="#fdb462", width=0.5),
            ),
            row=1,
            col=1,
        )

fig.add_trace(
            go.Scatter(
                x=validated_train_forecast_df.index,
                y=validated_train_forecast_df['price_second_auction_forecast'],
                mode="lines",
                name="Price train forecast second auction",
                line=dict(color="#80b1d3", width=0.5),
            ),
            row=1,
            col=2,
        )
            
fig.update_layout(title='Denormalized Training Actions Prices and Volumes')
fig.update_yaxes(title='Price', row=1)
fig.update_xaxes(title='Date', row=2)

fig.show()

In [ ]:

fig = make_subplots(
            rows=2,
            cols=2,
            shared_xaxes=True,
            vertical_spacing=0.08,
            subplot_titles=("1st Action Prices", "2nd Action Prices", "1st Action Volume", "2nd Action Volume"),
            row_width=[0.2, 0.7],
        )

fig.add_trace(
            go.Scatter(x=test_auction.index, y=test_auction["price_first_auction"], mode="lines", line_shape="spline",name="Test Price First Auction ", line=dict(color='#80b1d3')),
            row=1,
            col=1,
        )
fig.add_trace(
            go.Scatter(x=test_auction.index, y=test_auction["price_second_auction"], mode="lines", line_shape="spline",name="Test Price Second Auction", line=dict(color='#fb8072')),
            row=1,
            col=2,
        )

fig.add_trace(
            go.Bar(
                x=test_auction.index,
                y=test_auction["traded_volume_first_auction"],
                
                showlegend=False,
                marker=dict(
                    opacity=1,
                    line=dict(width=2, color="#ffffb3"),
                ),
            ),
            row=2,
            col=1,
        )

fig.add_trace(
            go.Bar(
                x=test_auction.index,
                y=test_auction["traded_volume_second_auction"],
                
                showlegend=False,
                marker=dict(
                    opacity=1,
                    line=dict(width=2, color="#ffffb3"),
                ),
            ),
            row=2,
            col=2,
        )

fig.add_trace(
            go.Scatter(
                x=test_auction.index,
                y=test_auction["price_forecast_first_auction"],
                mode="lines",
                name="Price train forecast first auction",
                line=dict(color="#fdb462", width=0.5),
            ),
            row=1,
            col=1,
        )

fig.add_trace(
            go.Scatter(
                x=validated_test_forecast_df.index,
                y=validated_test_forecast_df['price_second_auction_forecast'],
                mode="lines",
                name="Price train forecast second auction",
                line=dict(color="#80b1d3", width=0.5),
            ),
            row=1,
            col=2,
        )

fig.update_layout(title='Denormalized Test Actions Prices and Volumes', showlegend=True)
fig.update_yaxes(title='Price', row=1)
fig.update_xaxes(title='Date', row=2)

fig.show()

In [ ]:
start_date = pd.Timestamp('2022-02-28 23:00:00')
end_date = pd.Timestamp('2022-09-12 23:00:00')

# Calculate the number of hours between start_date and end_date
n_hours = (end_date - start_date).total_seconds() / 3600

print("Number of hours between start and end dates:", n_hours)



# **UNSEEN DATA SHORT TERM PREDICTIONS** 

The optimal time frame for precise forecasting is contingent upon the specific field and the quality of the data at hand. Within the realm of electricity price forecasting, short-term predictions (e.g., spanning a few hours to several days) tend to exhibit greater accuracy compared to their long-term counterparts (e.g., those extending over weeks or months). This is primarily due to the reduced uncertainty associated with shorter time frames. As evidenced by our attempt to forecast using previously unseen data, long-term projections (spanning 10 months) failed to align with the test data. In contrast, our initial models exhibited greater accuracy as they incorporated additional variables within the same time index, enabling the model to learn from contemporaneous data and generate more accurate predictions for the target variable.

Given the nature of the data, it appears prudent to concentrate on short-term forecasting, potentially within a 24-48 hour window. This time frame would enable the capture of daily trends and variations while maintaining a satisfactory level of precision. It is crucial to bear in mind that forecast accuracy typically declines as the prediction horizon expands. As such, it is essential to assess the performance of our forecasting model and recognize its limitations over extended periods.

Moving forward, our subsequent models will prioritize generating accurate short-term forecasts using previously unseen data, as would be the case in real-world trading scenarios. Subsequently, we will compare the predictive capabilities of various models to determine the most suitable approach for incorporating this data into our double auction strategies.

# XGBoost model: Predictions with Kfold validations 

In [ ]:


def extract_features_from_past_data(normalized_train_auction, normalized_train_forecast_inputs, normalized_train_system_prices):
    train_features = pd.concat([normalized_train_auction,
                                  normalized_train_forecast_inputs,
                                  normalized_train_system_prices], axis=1)
    return train_features

train_features = extract_features_from_past_data(normalized_train_auction, normalized_train_forecast_inputs, normalized_train_system_prices)

# print(train_features)

# Prepare the train data and target variable
train_data = train_features['price_second_auction']

# Number of hours we want to predict into the future
n_hours_for_prediction = 2 * 24

train_features = train_features[:-n_hours_for_prediction]
test_features = train_features[-n_hours_for_prediction:]

train_target = train_data[:-n_hours_for_prediction]
test_target = train_data[-n_hours_for_prediction:]

# Set up the K-Fold cross-validation
kf = KFold(n_splits=5, random_state=42, shuffle=True)

# Define the XGBoost model
xgbr = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=1000, learning_rate=0.05, max_depth=3, random_state=42)

mse_scores = []

# Perform cross-validation
for train_index, test_index in kf.split(train_features):
    X_train, X_test = train_features.iloc[train_index], train_features.iloc[test_index]
    y_train, y_test = train_target.iloc[train_index], train_target.iloc[test_index]
    
    # Train the model
    xgbr.fit(X_train, y_train)
    
    # Make predictions on the test set
    y_pred = xgbr.predict(X_test)
    
    # Calculate the mean squared error
    mse = mean_squared_error(y_test, y_pred)
    mse_scores.append(mse)

# Calculate the mean and standard deviation of the MSE scores from cross-validation
mean_mse = np.mean(mse_scores)
std_mse = np.std(mse_scores)

print("Mean MSE from Cross-Validation:", mean_mse)
print("Standard Deviation of MSE from Cross-Validation:", std_mse)

# Train the model on the entire dataset
xgbr.fit(train_features, train_target)

# Predict the next 2 * 24 hours
prediction = xgbr.predict(test_features)

# Assuming train_features has a DateTimeIndex
predict_period_dates = pd.date_range('2022-03-01', periods=n_hours_for_prediction, freq="H").tolist()

# Create a DataFrame with the forecast dates and predictions
future_forecast_df = pd.DataFrame({'DateTime': predict_period_dates, 'Prediction': prediction})

# print(future_forecast_df)

# Plot the predictions using Plotly
fig = go.Figure()

# Add a trace for the predictions
fig.add_trace(go.Scatter(x=future_forecast_df['DateTime'], y=future_forecast_df['Prediction'], mode='lines', name='Predictions'))

fig.update_layout(title='Future Price Predictions',
                  xaxis_title='Date',
                  yaxis_title='Price')

fig.show()



## Results plot: Normalized trin and comparison between forecast df made by XGBoost model with unseen data and the given test data 

In [ ]:
# Getting the mean and standard deviation used during normalization
mean = scaler.mean_[0]
std = scaler.scale_[0]

# Normalizing the train forecast
validated_train_forecast_df['price_second_auction_forecast'] = (validated_train_forecast_df['price_second_auction_forecast'] - mean) / std

# Normalizing the test forecast
validated_test_forecast_df['price_second_auction_forecast'] = (validated_test_forecast_df['price_second_auction_forecast'] - mean) / std


In [ ]:
print(future_forecast_df.index)

# Select data within the desired time range
train_start_date = '2022-02-26'
train_end_date = normalized_train_auction.index.max()
train_mask = (normalized_train_auction.index >= train_start_date) & (normalized_train_auction.index <= train_end_date)
train_df = normalized_train_auction.loc[train_mask]

validated_start_date = '2022-02-26'
validated_end_date = validated_train_forecast_df.index.max()
validated_mask = (validated_train_forecast_df.index >= validated_start_date) & (validated_train_forecast_df.index <= validated_end_date)
validated_df = validated_train_forecast_df.loc[validated_mask]

test_start_date = '2022-03-01'
test_end_date = '2022-03-04'
test_mask = (normalized_test_auction.index >= test_start_date) & (normalized_test_auction.index <= test_end_date)
test_df = normalized_test_auction.loc[test_mask]

forecast_mask = (future_forecast_df['DateTime'] >= test_start_date) & (future_forecast_df['DateTime'] <= test_end_date)
future_forecast_df = future_forecast_df.loc[forecast_mask]

# Create the subplots and add the traces
fig = make_subplots()

fig.add_trace(go.Scatter(x=train_df.index, y=train_df['price_second_auction'], name='Train Price Second Auction', line=dict(color='#80b1d3')))
fig.add_trace(go.Scatter(x=validated_df.index, y=validated_df['price_second_auction_forecast'], name='Train Price Second Auction Forecast', line=dict(color='#fdb462', width=0.8)))

fig.add_trace(go.Scatter(x=test_df.index, y=test_df['price_second_auction'], name='Test Price Second Auction', line=dict(color='#fb8072')))
fig.add_trace(go.Scatter(x=future_forecast_df['DateTime'], y=future_forecast_df['Prediction'], name='Test Price Second Auction Forecast', line=dict(color='#8dd3c7')))

# Update the layout
fig.update_layout(title='Normalized Train and Test Forecasts Second Auction')
fig.update_xaxes(title='Date')
fig.update_yaxes(title='Price')

# Show the plot
fig.show()


# SARIMAX
Regrettably, the current approach demands substantial system RAM and considerable processing time, without yielding improved predictions compared to our previous model. However, utilizing a GPU with increased memory capacity, potentially through cloud-based services, could streamline this process and facilitate the development of progressively superior models for our forecasting endeavors.

In [ ]:


# def extract_features_from_past_data(normalized_train_auction, normalized_train_forecast_inputs, normalized_train_system_prices):
#     train_features = pd.concat([normalized_train_auction,
#                                   normalized_train_forecast_inputs,
#                                   normalized_train_system_prices], axis=1)
#     return train_features

# train_features = extract_features_from_past_data(normalized_train_auction, normalized_train_forecast_inputs, normalized_train_system_prices)
# train_data = train_features['price_second_auction']

# warnings.filterwarnings("ignore")

# space = {
#     'p': hp.choice('p', range(0, 2)),
#     'd': hp.choice('d', range(0, 2)),
#     'q': hp.choice('q', range(0, 2)),
#     'P': hp.choice('P', range(0, 2)),
#     'D': hp.choice('D', range(0, 2)),
#     'Q': hp.choice('Q', range(0, 2)),
# }

# def objective(params, train_data_np):
#     model = SARIMAX(train_data_np, order=(params['p'], params['d'], params['q']), seasonal_order=(params['P'], params['D'], params['Q'], 24), enforce_stationarity=False, enforce_invertibility=False)
#     try:
#         results = model.fit()
#         return results.aic
#     except:
#         return float('inf')

# train_data_np = train_data.to_numpy()

# best = fmin(partial(objective, train_data_np=train_data_np), space, algo=tpe.suggest, max_evals=50)

# best_pdq = (best['p'], best['d'], best['q'])
# best_seasonal_pdq = (best['P'], best['D'], best['Q'], 24)

# print(f"Best SARIMA model parameters: {best_pdq}x{best_seasonal_pdq}")

# sarima_model = SARIMAX(train_data, order=best_pdq, seasonal_order=best_seasonal_pdq, enforce_stationarity=False, enforce_invertibility=False)
# sarima_results = sarima_model.fit()

# n_hours_for_prediction = 2 * 24
# prediction = sarima_results.get_forecast(steps=n_hours_for_prediction).predicted_mean

# predict_period_dates = pd.date_range(train_features.index[-1] + pd.Timedelta(hours=1), periods=n_hours_for_prediction, freq="H").tolist()

# future_forecast_df2 = pd.DataFrame({'DateTime': predict_period_dates, 'Prediction': prediction})

# fig = go.Figure()
# fig.add_trace(go.Scatter(x=future_forecast_df2['DateTime'], y=future_forecast_df2['Prediction'], mode='lines', name='Predictions'))

# fig.update_layout(title='Future Price Predictions',
#                   xaxis_title='Date',
#                   yaxis_title='Price')

# fig.show()


## Result Sarimax plot: normalized train data and comparison between test data and the Sarimax forecast df made with unseen data

In [ ]:

# # Select data within the desired time range
# train_start_date = '2022-02-26'
# train_end_date = normalized_train_auction.index.max()
# train_mask = (normalized_train_auction.index >= train_start_date) & (normalized_train_auction.index <= train_end_date)
# train_df = normalized_train_auction.loc[train_mask]

# validated_start_date = '2022-02-26'
# validated_end_date = validated_train_forecast_df.index.max()
# validated_mask = (validated_train_forecast_df.index >= validated_start_date) & (validated_train_forecast_df.index <= validated_end_date)
# validated_df = validated_train_forecast_df.loc[validated_mask]

# test_start_date = '2022-03-01'
# test_end_date = '2022-03-04'
# test_mask = (normalized_test_auction.index >= test_start_date) & (normalized_test_auction.index <= test_end_date)
# test_df = normalized_test_auction.loc[test_mask]

# forecast_mask = (future_forecast_df2['DateTime'] >= test_start_date) & (future_forecast_df2['DateTime'] <= test_end_date)
# future_forecast_df2 = future_forecast_df2.loc[forecast_mask]

# # Create the subplots and add the traces
# fig = make_subplots()

# fig.add_trace(go.Scatter(x=train_df.index, y=train_df['price_second_auction'], name='Train Price Second Auction', line=dict(color='#80b1d3')))
# fig.add_trace(go.Scatter(x=validated_df.index, y=validated_df['price_second_auction_forecast'], name='Train Price Second Auction Forecast', line=dict(color='#fdb462', width=0.8)))

# fig.add_trace(go.Scatter(x=test_df.index, y=test_df['price_second_auction'], name='Test Price Second Auction', line=dict(color='#fb8072')))
# fig.add_trace(go.Scatter(x=future_forecast_df2['DateTime'], y=future_forecast_df2['Prediction'], name='Test Price Second Auction Forecast', line=dict(color='#8dd3c7')))

# # Update the layout
# fig.update_layout(title='Normalized Train and Test Forecasts Second Auction')
# fig.update_xaxes(title='Date')
# fig.update_yaxes(title='Price')

# # Show the plot
# fig.show()


# **TRADING ALGORITHM**

Upon evaluating the outcomes of various models, the XGBoost and SARIMAX models emerge as the top performers in terms of runtime, RAM usage, computational efficiency, and accuracy. Among these two contenders, the XGBoost model demonstrates superior precision, and as such, we will utilize its predictive values to inform our strategic approach.

We assume that the "future data" represents the test data prices for each auction, with the initial price obtained from the forecast of the first auction already provided. For the second price, we will employ the future_forecast_df dataframe, derived from the XGBoost model, under the pretense that the test data was unseen, thereby allowing for an accuracy comparison. This information will be utilized to determine the appropriate bids and volumes, which will then be integrated into our trading algorithm.

In [ ]:
# Importing libraries
import random
import pandas as pd
from typing import List, Tuple

## Organizing input data

In [ ]:
# First we need to denormalize the XGBoost model results.

# Getting the mean and standard deviation used during normalization
mean = scaler.mean_[0]
std = scaler.scale_[0]

# Denormalizing the future forecast
future_forecast_df['Prediction'] = future_forecast_df['Prediction'] * std + mean
print(f"{future_forecast_df}")

test_auction = test_auction[test_auction.index < '2022-03-02 23:00:00']
print(test_auction["price_forecast_first_auction"])

test_forecast_inputs = test_forecast_inputs[test_forecast_inputs.index < '2022-03-02 23:00:00']
print(f"test_forecast_inputs {test_forecast_inputs.columns}")

test_system_prices = test_system_prices[test_system_prices.index < '2022-03-02 23:00:00']
print(f"test_system_prices {test_system_prices.columns}")

test_auction = test_auction[test_auction.index < '2022-03-02 23:00:00']
print(f"test_auction {test_auction.columns}")


## Calculate the bids and the volume

In [ ]:
class BidStrategy:
    def __init__(self, train_auction, train_system_prices, train_forecast_inputs, test_auction, test_system_prices, test_forecast_inputs, future_forecast_df):
        self.train_auction = train_auction
        self.train_system_prices = train_system_prices
        self.train_forecast_inputs = train_forecast_inputs
        self.test_auction = test_auction
        self.test_system_prices = test_system_prices
        self.test_forecast_inputs = test_forecast_inputs
        self.future_forecast_df = future_forecast_df

    def analyze_data(self):
        # Analyze historical data to identify trends and patterns
        # You can add logic to process the train_auction, train_system_prices, and train_forecast_inputs dataframes
        pass

    def predict_future_prices(self):
        # Use forecasted data to predict short-term price movements
        # You can add logic to process the test_auction and future_forecast_df dataframes
        pass

    def calculate_bids(self):
        self.analyze_data()
        self.predict_future_prices()

        # Calculating clearing prices for each auction
        clear_price_1 = self.test_auction['price_forecast_first_auction'].max()
        clear_price_2 = self.future_forecast_df['Prediction'].max()

        # Calculating payments for each bidder based on their bids
        payment_1 = clear_price_1 - self.test_system_prices['system_price'].values
        payment_2 = clear_price_2 - self.test_system_prices['system_price'].values

        # Calculating revenues for each bidder based on their payments and winning quantity
        revenue_1 = payment_1 * self.test_forecast_inputs['demand'].values
        revenue_2 = payment_2 * self.test_forecast_inputs['demand'].values

        # Calculating the traded volume for each bidder based on their revenue and clearing price
        volume_1 = revenue_1 / clear_price_1
        volume_2 = revenue_2 / clear_price_2

        # Calculating the bid price and traded volume for each bidder
        bid_price_1 = clear_price_1 - (revenue_1 / volume_1)
        bid_price_2 = clear_price_2 - (revenue_2 / volume_2)
        traded_volume_1 = self.test_auction['traded_volume_first_auction'].values
        traded_volume_2 = self.test_auction['traded_volume_second_auction'].values

        # Returning hourly bids as a list of tuples with traded volume and bid price
        hourly_bids = [(int(traded_volume_1[i]), bid_price_1[i]) for i in range(len(bid_price_1))]
        hourly_bids += [(int(traded_volume_2[i]), bid_price_2[i]) for i in range(len(bid_price_2))]
        return hourly_bids

# Instantiating the BidStrategy class 
bid_strategy = BidStrategy(train_auction, train_system_prices, train_forecast_inputs, test_auction, test_system_prices, test_forecast_inputs, future_forecast_df)

bids = bid_strategy.calculate_bids()

print(bids)


## Trading tool and further analysis

In [ ]:


class Auction:
    def __init__(self, train_auction, future_forecast_df, threshold=0.1):
        self.train_auction = train_auction
        self.future_forecast_df = future_forecast_df
        self.threshold = threshold
        self.bids = []

    def analyze_predicted_prices(self):
        # Calculate the price differences between the two auctions
        self.train_auction["price_difference"] = self.future_forecast_df["Predictions"] - self.train_auction["price_forecast_first_auction"]
        self.train_auction["price_difference_percentage"] = self.train_auction["price_difference"] / self.train_auction["price_forecast_first_auction"]
        
        # Identify opportunities for arbitrage
        self.train_auction["arbitrage_opportunity"] = self.train_auction["price_difference_percentage"].apply(lambda x: x > self.threshold)
        
        return self.train_auction

    def place_bids(self):
        analyzed_prices = self.analyze_predicted_prices()
        for _, row in analyzed_prices.iterrows():
            if row["arbitrage_opportunity"]:
                bid = (row["volume"], row["price_forecast_first_auction"])
            else:
                bid = (0, 0)
            self.bids.append(bid)

    def price_range_system_price(self) -> Tuple[float, float]:
        return (-10, 10)

    def calculate_system_price(self, system_price_range: Tuple[float, float]) -> float:
        return random.uniform(system_price_range[0], system_price_range[1])

    def price_auction(self, bids: List[Tuple[float, float]]) -> Tuple[float, float]:
        if bids:
            winning_price = max(bid[1] for bid in bids)
            net_position = sum(bid[0] for bid in bids if bid[1] >= winning_price)
        else:
            winning_price = 0
            net_position = 0
        return (winning_price, net_position)

    def calculate_profit_loss(self, net_positions: List[float], winning_prices: List[float], system_price: float) -> List[float]:
        profit_loss = []
        for i in range(len(net_positions)):
            trade_cost = abs(net_positions[i]) * 5
            net_position_value = net_positions[i] * winning_prices[i] - trade_cost

            if net_positions[i] < 0:
                net_position_value += abs(net_positions[i]) * system_price
            elif net_positions[i] > 0:
                net_position_value -= net_positions[i] * system_price

            profit_loss.append(net_position_value)
        return profit_loss

    def submit_bids(self, bids: List[Tuple[float, float]]):
        self.bids.append(bids)

    def run_auctions(self):
        system_price_range = self.price_range_system_price()
        system_price = self.calculate_system_price(system_price_range)

        winning_prices = []
        net_positions = []

        for hourly_bids in self.bids:
            winning_price, net_position = self.price_auction(hourly_bids)
            winning_prices.append(winning_price)
            net_positions.append(net_position)

        profit_loss = self.calculate_profit_loss(net_positions, winning_prices, system_price)
        return winning_prices, net_positions, profit_loss






In [ ]:
auction = Auction()

# Submit bids for each hour of the next operating day
hourly_bids = [
    (10, 20), (20, 30), (30, 25), (25, 20)
]

for hour in range(24):
    auction.submit_bids(hourly_bids)

# First auction results
winning_prices1, net_positions1, _ = auction.run_auctions()

# Second auction bids
bids_second_auction = [(-bid[0], bid[1]) for bid in hourly_bids]

auction = Auction()
for hour in range(24):
    auction.submit_bids(bids_second_auction)

# Second auction results
winning_prices2, net_positions2, _ = auction.run_auctions()

# Calculate net positions and profit/loss
net_positions = [net1 + net2 for net1, net2 in zip(net_positions1, net_positions2)]
winning_prices = [(price1 + price2) / 2 for price1, price2 in zip(winning_prices1, winning_prices2)]
system_price_range = auction.price_range_system_price()
system_price = auction.calculate_system_price(system_price_range)
profit_loss = auction.calculate_profit_loss(net_positions, winning_prices, system_price)

print(f"Winning prices: {winning_prices}")
print(f"Net positions: {net_positions}")
print(f"Profit/loss for both auctions: {profit_loss}")